In [ ]:
# Google Colab環境の設定とライブラリインポート
import os

# プロジェクトのルートディレクトリに移動
if '/content/sakata_goho_chart' not in os.getcwd():
    os.chdir('/content/sakata_goho_chart')

print(f"現在のディレクトリ: {os.getcwd()}")

# 必要なライブラリのインポート
import sys
sys.path.append('.')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 自作モジュールのインポート
from src.data_fetcher import DataFetcher
from src.technical_indicators import TechnicalIndicators
from src.utils import DataPreprocessor, ModelUtils

# Transformerモデル用のライブラリ
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout
from tensorflow.keras.layers import MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

print("ライブラリのインポート完了")


In [ ]:
# データ取得
from google.colab import userdata

# KuCoin APIの設定
fetcher = DataFetcher(
    exchange_name='kucoin',
    api_key=userdata.get('KuCoin_API_KEY'),
    secret=userdata.get('KuCoin_API_SECRET'),
    password=userdata.get('KuCoin_API_PASSPHRAS')
)

# データ取得設定
symbol = 'SOL/USDT'
timeframe = '1d'
limit = 8760 * 2  # 2年分

print(f"データ取得開始: {symbol}")
data = fetcher.fetch_ohlcv_data(symbol, timeframe, limit)

if data is not None:
    print(f"取得完了: {len(data)}件")
    print(data.head())
    
    # Google Driveに保存（バックアップ用）
    backup_path = "/content/drive/MyDrive/trading_bot_data.csv"
    data.to_csv(backup_path)
    print(f"データをGoogle Driveに保存: {backup_path}")
else:
    print("データ取得に失敗しました。APIキーを確認してください。")


In [ ]:
# テクニカル指標の追加
print("テクニカル指標を計算中...")
df = TechnicalIndicators.add_all_indicators(data)
print(f"テクニカル指標追加完了: {df.shape}")

# データの概要を表示
print("\nデータの基本統計:")
print(df.describe())

# データを可視化
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.plot(df.index, df['Close'])
plt.title('Price Chart')
plt.ylabel('Price')

plt.subplot(2, 2, 2)
plt.plot(df.index, df['RSI'])
plt.title('RSI')
plt.ylabel('RSI')
plt.axhline(y=70, color='r', linestyle='--', alpha=0.5)
plt.axhline(y=30, color='r', linestyle='--', alpha=0.5)

plt.subplot(2, 2, 3)
plt.plot(df.index, df['MACD'])
plt.title('MACD')
plt.ylabel('MACD')

plt.subplot(2, 2, 4)
plt.plot(df.index, df['Volume'])
plt.title('Volume')
plt.ylabel('Volume')

plt.tight_layout()
plt.show()

df.head()


In [ ]:
# Transformerモデルの定義（Google Colab用）
def create_transformer_model(input_dim, num_heads=8, ff_dim=32, num_layers=2, dropout_rate=0.1):
    """
    Transformerモデルを作成
    
    Args:
        input_dim: 入力次元数
        num_heads: マルチヘッドアテンションのヘッド数
        ff_dim: フィードフォワード層の次元数
        num_layers: Transformerレイヤーの数
        dropout_rate: ドロップアウト率
    
    Returns:
        Kerasモデル
    """
    inputs = Input(shape=(1, input_dim))
    
    # Transformerブロック
    x = inputs
    for _ in range(num_layers):
        # Multi-Head Attention
        attention_output = MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=input_dim // num_heads
        )(x, x)
        attention_output = Dropout(dropout_rate)(attention_output)
        x = LayerNormalization(epsilon=1e-6)(x + attention_output)
        
        # Feed Forward
        ffn_output = Dense(ff_dim, activation="relu")(x)
        ffn_output = Dense(input_dim)(ffn_output)
        ffn_output = Dropout(dropout_rate)(ffn_output)
        x = LayerNormalization(epsilon=1e-6)(x + ffn_output)
    
    # Global Average Pooling
    x = GlobalAveragePooling1D()(x)
    
    # 出力層
    x = Dense(64, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

print("Transformerモデル定義完了")


In [ ]:
# データの前処理
print("データの前処理開始...")
preprocessor = DataPreprocessor(scaler_type='standard')
X_scaled, y = preprocessor.fit_transform(df, target_column='Close')

# 最後の行は予測対象なので除外
X_scaled = X_scaled[:-1]
y = y[:-1]

print(f"特徴量数: {X_scaled.shape[1]}")
print(f"データ数: {X_scaled.shape[0]}")
print(f"ターゲット分布: {np.bincount(y)}")

# 学習・テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Transformerモデル用にデータを3次元に変換
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

print(f"訓練データ: {X_train.shape}")
print(f"テストデータ: {X_test.shape}")

# GPU使用状況の確認
print(f"\nGPU使用可能: {tf.config.list_physical_devices('GPU')}")
if tf.config.list_physical_devices('GPU'):
    print("✅ GPUが使用可能です")
else:
    print("⚠️ CPUで実行されます")


In [ ]:
# モデルの作成とコンパイル
print("モデルの作成とコンパイル...")
model = create_transformer_model(input_dim=X_train.shape[2])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# モデルの概要を表示
model.summary()

# Google Colabでの訓練進捗を監視するコールバック
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Google Driveにモデルを保存するパス
checkpoint_path = "/content/drive/MyDrive/transformer_model_checkpoint.h5"

callbacks = [
    ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("モデル準備完了")


In [ ]:
# モデルの訓練開始
print("🚀 モデル訓練を開始します...")
print("⏱️ 訓練時間は約15-30分程度を予想しています")

import time
start_time = time.time()

history = model.fit(
    X_train, y_train,
    epochs=100,  # 多めに設定（EarlyStoppingで自動停止）
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

end_time = time.time()
training_time = end_time - start_time

print(f"✅ 訓練完了！")
print(f"⏱️ 訓練時間: {training_time/60:.1f}分")

# 訓練結果の可視化
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training Loss', alpha=0.8)
plt.plot(history.history['val_loss'], label='Validation Loss', alpha=0.8)
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', alpha=0.8)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', alpha=0.8)
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.plot(history.history['lr'] if 'lr' in history.history else [], label='Learning Rate', alpha=0.8)
plt.title('Learning Rate')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# モデルの評価と保存
print("📊 モデルの評価中...")

# テストデータでの評価
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ テスト精度: {test_accuracy:.4f}")
print(f"📉 テスト損失: {test_loss:.4f}")

# 予測結果の詳細分析
y_pred = model.predict(X_test, verbose=0)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report, confusion_matrix
print("\n📋 分類レポート:")
print(classification_report(y_test, y_pred_binary))

print("\n🎯 混同行列:")
cm = confusion_matrix(y_test, y_pred_binary)
print(cm)

# 混同行列の可視化
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
classes = ['下落', '上昇']
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)

# 数値を表示
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel('実際のラベル')
plt.xlabel('予測ラベル')
plt.tight_layout()
plt.show()

# Google Driveにモデルとスケーラーを保存
final_model_path = "/content/drive/MyDrive/transformer_model_final.h5"
scaler_path = "/content/drive/MyDrive/scaler.pkl"

model.save(final_model_path)
preprocessor.save_scaler(scaler_path)

print(f"💾 最終モデルを保存: {final_model_path}")
print(f"💾 スケーラーを保存: {scaler_path}")

# 訓練サマリーの保存
import json
training_summary = {
    'symbol': symbol,
    'timeframe': timeframe,
    'data_points': len(data),
    'features': X_train.shape[2],
    'training_time_minutes': training_time/60,
    'final_accuracy': float(test_accuracy),
    'final_loss': float(test_loss),
    'epochs_trained': len(history.history['loss'])
}

with open("/content/drive/MyDrive/training_summary.json", "w") as f:
    json.dump(training_summary, f, indent=2)

print("📝 訓練サマリーを保存しました")
print("\n🎉 全ての処理が完了しました！")
